In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, roc_auc_score, confusion_matrix,
                           classification_report)
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

In [2]:

class SalesRiskPredictor:
    def __init__(self):
        self.models = {
            'rf': RandomForestClassifier(
                n_estimators=100,
                max_depth=5,
                class_weight='balanced',
                random_state=42
            ),
            'gb': GradientBoostingClassifier(
                n_estimators=100,
                max_depth=3,
                random_state=42
            )
        }
        self.scaler = StandardScaler()
        self.feature_names = None

    def create_risk_labels(self, df):
        """Create risk labels using multiple criteria"""
        risk_factors = pd.DataFrame()

        # 1. Opportunity decline
        risk_factors['opp_decline'] = (
            df['Opportunity Created (last month)'] < df['Opportunity Created (month before last)']
        )

        # 2. Activity decline (20% or more decline in calls or emails)
        risk_factors['call_decline'] = (
            df['Outbound Calls (last month)'] < df['Outbound Calls (month before last)'] * 0.8
        )

        risk_factors['email_decline'] = (
            df['Personalized Outbound Emails (last month)'] <
            df['Personalized Outbound Emails (month before last)'] * 0.8
        )

        # 3. Conversion rate decline
        curr_calls = df['Outbound Calls (last month)'].replace(0, 1)
        prev_calls = df['Outbound Calls (month before last)'].replace(0, 1)
        curr_conv = df['Opportunity Created (last month)'] / curr_calls
        prev_conv = df['Opportunity Created (month before last)'] / prev_calls
        risk_factors['conv_decline'] = curr_conv < prev_conv * 0.8

        # 4. Below median current performance
        median_opps = df['Opportunity Created (last month)'].median()
        risk_factors['below_median'] = df['Opportunity Created (last month)'] < median_opps

        # Calculate total risk factors
        total_risk_factors = risk_factors.sum(axis=1)

        # Label as at-risk if meeting two or more risk factors
        return (total_risk_factors >= 2).astype(int)

    def prepare_features(self, df):
        """Prepare features for model training"""
        features = pd.DataFrame()

        # Activity Metrics
        metrics = [
            'Outbound Calls',
            'Personalized Outbound Emails',
            'Calls with Correct Contact',
            'Demo Meeting Set',
            'Demo Meeting Completed',
            'Opportunity Created'
        ]

        for metric in metrics:
            current = df[f'{metric} (last month)']
            previous = df[f'{metric} (month before last)']

            features[f'{metric}_current'] = current
            features[f'{metric}_previous'] = previous

            # Calculate changes
            features[f'{metric}_change'] = (
                (current - previous) / previous.replace(0, 1)
            ).clip(-1, 1)

            # Relative to median
            median_curr = current.median()
            if median_curr != 0:
                features[f'{metric}_vs_median'] = (current - median_curr) / median_curr
            else:
                features[f'{metric}_vs_median'] = 0

        # Clean data
        features = features.replace([np.inf, -np.inf], 0)
        features = features.fillna(0)

        self.feature_names = features.columns.tolist()
        return features

    def analyze_individual_risk(self, df, employee_data, model_name='rf'):
        """Analyze risk factors for specific employee"""
        # Prepare features
        features = self.prepare_features(pd.DataFrame([employee_data]))
        scaled_features = self.scaler.transform(features)

        # Get prediction
        model = self.models[model_name]
        risk_proba = model.predict_proba(scaled_features)[0, 1]

        # Calculate changes and percentages
        def calc_change(current, previous):
            if previous == 0:
                return 0 if current == 0 else 100
            return ((current - previous) / previous) * 100

        def calc_percentile(series, value):
            return stats.percentileofscore(series.dropna(), value)

        metrics_analysis = {
            'opportunities': {
                'current': employee_data['Opportunity Created (last month)'],
                'previous': employee_data['Opportunity Created (month before last)'],
                'change_pct': calc_change(
                    employee_data['Opportunity Created (last month)'],
                    employee_data['Opportunity Created (month before last)']
                )
            },
            'calls': {
                'current': employee_data['Outbound Calls (last month)'],
                'previous': employee_data['Outbound Calls (month before last)'],
                'change_pct': calc_change(
                    employee_data['Outbound Calls (last month)'],
                    employee_data['Outbound Calls (month before last)']
                )
            },
            'emails': {
                'current': employee_data['Personalized Outbound Emails (last month)'],
                'previous': employee_data['Personalized Outbound Emails (month before last)'],
                'change_pct': calc_change(
                    employee_data['Personalized Outbound Emails (last month)'],
                    employee_data['Personalized Outbound Emails (month before last)']
                )
            }
        }

        # Peer comparisons
        peer_metrics = {
            'opportunities': {
                'employee': employee_data['Opportunity Created (last month)'],
                'team_median': df['Opportunity Created (last month)'].median(),
                'percentile': calc_percentile(
                    df['Opportunity Created (last month)'],
                    employee_data['Opportunity Created (last month)']
                )
            },
            'calls': {
                'employee': employee_data['Outbound Calls (last month)'],
                'team_median': df['Outbound Calls (last month)'].median(),
                'percentile': calc_percentile(
                    df['Outbound Calls (last month)'],
                    employee_data['Outbound Calls (last month)']
                )
            },
            'emails': {
                'employee': employee_data['Personalized Outbound Emails (last month)'],
                'team_median': df['Personalized Outbound Emails (last month)'].median(),
                'percentile': calc_percentile(
                    df['Personalized Outbound Emails (last month)'],
                    employee_data['Personalized Outbound Emails (last month)']
                )
            }
        }

        # Feature importance specific to this employee
        feature_importance = pd.DataFrame({
            'feature': self.feature_names,
            'importance': model.feature_importances_,
            'value': features.iloc[0].values
        })

        return {
            'name': employee_data['user_name'],
            'risk_score': risk_proba,
            'risk_level': 'High' if risk_proba >= 0.7 else 'Medium' if risk_proba >= 0.4 else 'Low',
            'metrics': metrics_analysis,
            'peer_comparison': peer_metrics,
            'feature_importance': feature_importance.sort_values('importance', ascending=False).to_dict('records')
        }

    def train_evaluate_models(self, df):
        """Train and evaluate risk prediction models with cross validation"""
        X = self.prepare_features(df)
        y = self.create_risk_labels(df)

        # Print class distribution
        class_counts = np.bincount(y)
        print(f"\nClass distribution:")
        print(f"Not at risk: {class_counts[0]}")
        print(f"At risk: {class_counts[1]}")

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        results = {}
        for name, model in self.models.items():
            # Cross-validation
            cv = KFold(n_splits=5, shuffle=True, random_state=42)
            cv_results = cross_validate(
                model, X_train_scaled, y_train,
                cv=cv,
                scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
                return_train_score=True
            )

            # Train final model
            model.fit(X_train_scaled, y_train)

            # Feature importance
            importance = pd.DataFrame({
                'feature': self.feature_names,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)

            results[name] = {
                'cv_results': {
                    metric: {
                        'mean': cv_results[f'test_{metric}'].mean(),
                        'std': cv_results[f'test_{metric}'].std()
                    } for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
                },
                'feature_importance': {
                    'features': importance.to_dict('records')
                }
            }

        return results

    def predict_risk(self, df, model_name='rf'):
        """Generate risk predictions"""
        X = self.prepare_features(df)
        X_scaled = self.scaler.transform(X)

        model = self.models[model_name]
        risk_proba = model.predict_proba(X_scaled)[:, 1]

        return pd.DataFrame({
            'user_name': df['user_name'],
            'risk_score': risk_proba,
            'risk_level': np.where(risk_proba >= 0.7, 'High',
                                 np.where(risk_proba >= 0.4, 'Medium', 'Low'))
        })

def format_insights(analysis):
    """Format analysis insights for display"""
    metrics = analysis['metrics']
    peers = analysis['peer_comparison']
    insights = []

    # Performance changes
    for metric, data in metrics.items():
        if data['change_pct'] < -20:
            insights.append(f"{metric.title()} declined by {abs(data['change_pct']):.1f}% "
                          f"(from {data['previous']} to {data['current']})")

    # Peer comparisons
    for metric, data in peers.items():
        if data['percentile'] < 25:
            insights.append(f"{metric.title()} in bottom quartile "
                          f"({data['employee']} vs team median {data['team_median']:.1f})")

    return insights

In [4]:
def main():
    try:
        # Load and process data
        print("Loading data...")
        df = pd.read_csv('monthly_report_expanded.csv')

        predictor = SalesRiskPredictor()

        # Train models and generate predictions
        print("Training models...")
        model_results = predictor.train_evaluate_models(df)

        print("Generating predictions...")
        risk_predictions = predictor.predict_risk(df)

        # Save high-risk reps to CSV
        high_risk_reps = risk_predictions[risk_predictions['risk_level'] == 'High']
        high_risk_reps.to_csv('high_risk_reps.csv', index=False)
        print(f"\nSaved {len(high_risk_reps)} high-risk representatives to 'high_risk_reps.csv'")

        # Display model performance metrics
        print("\nModel Performance (Random Forest):")
        rf_results = model_results['rf']
        for metric, values in rf_results['cv_results'].items():
            print(f"{metric}: {values['mean']:.3f} (±{values['std']:.3f})")

        # Display feature importance
        print("\nTop 5 Important Features:")
        for feature in rf_results['feature_importance']['features'][:5]:
            print(f"{feature['feature']}: {feature['importance']:.3f}")

        # Display risk distribution
        print(f"\nRisk Level Distribution:")
        print(risk_predictions['risk_level'].value_counts())
        print("\nRisk Level Percentages:")
        print(risk_predictions['risk_level'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%')

        # Get examples from each risk category
        risk_levels = {'High': None, 'Medium': None, 'Low': None}
        for level in risk_levels:

            mask = risk_predictions['risk_level'] == level
            if mask.any():
                employee = risk_predictions[mask].iloc[0]
                emp_data = df[df['user_name'] == employee['user_name']].iloc[0]
                analysis = predictor.analyze_individual_risk(df, emp_data)
                risk_levels[level] = analysis

      
        print("\n=== Detailed Risk Analysis ===")
        for level, analysis in risk_levels.items():
            if analysis:
                print("Risk Factor: ", level)
                print(f"\nExample {level} Risk Employee:")
                print(f"Name: {analysis['name']}")
                print(f"Risk Score: {analysis['risk_score']:.3f}")

                print("\nKey Risk Factors:")
                insights = format_insights(analysis)
                for insight in insights:
                    print(f"- {insight}")

                print("\nTop Contributing Features:")
                for feature in analysis['feature_importance'][:3]:
                    print(f"- {feature['feature']}: {feature['importance']:.3f}")

                print("\nPeer Comparison Summary:")
                for metric, data in analysis['peer_comparison'].items():
                    print(f"- {metric.title()}: {data['employee']} "
                          f"(Team Median: {data['team_median']:.1f}, "
                          f"Percentile: {data['percentile']:.1f})")
                print("-" * 50)


    except Exception as e:
        print(f"Error in analysis: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Loading data...
Training models...

Class distribution:
Not at risk: 42
At risk: 8
Generating predictions...

Saved 6 high-risk representatives to 'high_risk_reps.csv'

Model Performance (Random Forest):
accuracy: 0.925 (±0.061)
precision: 0.400 (±0.490)
recall: 0.300 (±0.400)
f1: 0.333 (±0.422)
roc_auc: nan (±nan)

Top 5 Important Features:
Outbound Calls_change: 0.282
Demo Meeting Completed_change: 0.094
Outbound Calls_vs_median: 0.076
Calls with Correct Contact_current: 0.066
Outbound Calls_current: 0.054

Risk Level Distribution:
risk_level
Low       42
High       6
Medium     2
Name: count, dtype: int64

Risk Level Percentages:
risk_level
Low       84.0%
High      12.0%
Medium     4.0%
Name: proportion, dtype: object

=== Detailed Risk Analysis ===
Risk Factor:  High

Example High Risk Employee:
Name: Lisa Park
Risk Score: 0.688

Key Risk Factors:
- Calls declined by 20.9% (from 153 to 121)
- Calls in bottom quartile (121 vs team median 205.5)

Top Contributing Features:
- Outboun